In [22]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import cv2
import open3d as o3d
from find_box import get_box_mask, get_box_pc, prepare_pc, extract_box_pc, estimate_box_pose
from realsense_camera import RealsenseCamera
from camera_utils import stream
from copy import deepcopy

def show(image):
    cv2.imshow("image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# initialization

camera = RealsenseCamera(enable_depth=True)

camera.start()
K = camera.get_K()
depth_scale = camera.get_depth_scale()
camera.stop()

voxel_size = 0.003
box_pc = get_box_pc([0.11, 0.05, 0.022])
box_pc_down, box_fpfh = prepare_pc(box_pc, voxel_size=voxel_size)

In [24]:
# test box segmentation

camera.start()
stream(camera)
image = camera()
camera.stop()

mask = get_box_mask(image)
masked_image = image.copy()
masked_image[mask == 0] = 0
show(masked_image)

In [28]:
# estimate box pose

camera.start()
stream(camera)
image, depth = camera.read(read_color=True, read_depth=True)
camera.stop()

voxel_size = 0.003

extracted_box_pc = extract_box_pc(image, depth, depth_scale, K)
if len(extracted_box_pc.points) < 1000:
    print("Cloud not extract box point cloud")
else:
    extracted_box_pc_down, extracted_box_fpfh = prepare_pc(extracted_box_pc, voxel_size=voxel_size)

    global_reg, reg = estimate_box_pose(
        extracted_box_pc, box_pc,
        extracted_box_pc_down, box_pc_down,
        extracted_box_fpfh, box_fpfh,
        voxel_size=0.025)

    box_pose = reg.transformation

In [26]:
# point clouds visualization

transformed_box_pc = deepcopy(box_pc)
transformed_box_pc.transform(box_pose)
transformed_box_pc.paint_uniform_color(np.array([1, 0, 0]))
extracted_box_pc.paint_uniform_color(np.array([0, 1, 0]))
o3d.visualization.draw_geometries([transformed_box_pc, extracted_box_pc])